In [10]:
from datetime import datetime, timedelta
from typing import Optional, Tuple

# =================================================================
# 1. BASE CLASS (Inheritance)
# =================================================================
class LibraryItem:
    """Base class for all library items."""

    def __init__(self, title: str, isbn: str, year: int):
        self.title = title
        self.isbn = isbn
        self.year = year
        self.available = True

    def check_out(self) -> Tuple[Optional[str], str]:
        """
        Polymorphic method to be overridden by derived classes.
        Returns (due_date, message) or (None, error_message)
        """
        if not self.available:
            return None, f"'{self.title}' is already checked out."
        self.available = False
        return None, "Generic checkout (override in subclass)"

    def return_item(self):
        """Return the item to available status."""
        self.available = True

    def __str__(self):
        status = "Available" if self.available else "Checked Out"
        return f"[{status}] {self.title} ({self.isbn})"


# =================================================================
# 2. DERIVED CLASSES (Inheritance + Polymorphism)
# =================================================================
class Book(LibraryItem):
    """Book with 14-day loan period."""

    def __init__(self, title: str, isbn: str, year: int, author: str, genre: str):
        super().__init__(title, isbn, year)
        self.author = author
        self.genre = genre

    def check_out(self) -> Tuple[Optional[str], str]:
        if not self.available:
            return None, f"'{self.title}' is already checked out."

        self.available = False
        due_date = (datetime.now() + timedelta(days=14)).strftime("%Y-%m-%d")
        return due_date, f"📚 Book '{self.title}' checked out. Due: {due_date}."

    def __str__(self):
        status = "Available" if self.available else "Checked Out"
        return f"[{status}] 📚 Book: {self.title} by {self.author} ({self.year})"


class DVD(LibraryItem):
    """DVD with 3-day loan period."""

    def __init__(self, title: str, isbn: str, year: int, director: str):
        super().__init__(title, isbn, year)
        self.director = director

    def check_out(self) -> Tuple[Optional[str], str]:
        if not self.available:
            return None, f"'{self.title}' is already checked out."

        self.available = False
        due_date = (datetime.now() + timedelta(days=3)).strftime("%Y-%m-%d")
        return due_date, f"📀 DVD '{self.title}' checked out. Due: {due_date}."

    def __str__(self):
        status = "Available" if self.available else "Checked Out"
        return f"[{status}] 📀 DVD: {self.title} directed by {self.director} ({self.year})"


class EBook(LibraryItem):
    """EBook with 28-day loan period."""

    def __init__(self, title: str, isbn: str, year: int, author: str, file_size_mb: float):
        super().__init__(title, isbn, year)
        self.author = author
        self.file_size_mb = file_size_mb

    def check_out(self) -> Tuple[Optional[str], str]:
        if not self.available:
            return None, f"'{self.title}' is already checked out."

        self.available = False
        due_date = (datetime.now() + timedelta(days=28)).strftime("%Y-%m-%d")
        return due_date, f"📱 EBook '{self.title}' checked out. Due: {due_date}."

    def __str__(self):
        status = "Available" if self.available else "Checked Out"
        return f"[{status}] 📱 EBook: {self.title} by {self.author} ({self.file_size_mb}MB)"


# =================================================================
# 3. COMPOSITION CLASSES
# =================================================================
class LibraryCatalog:
    """Manages the collection of library items."""

    def __init__(self):
        self._items = {}

    def add_item(self, item: LibraryItem):
        """Add an item to the catalog."""
        self._items[item.isbn] = item

    def get_item(self, isbn: str) -> Optional[LibraryItem]:
        """Retrieve an item by ISBN."""
        return self._items.get(isbn)

    @property
    def all_items(self):
        """Return all items in the catalog."""
        return list(self._items.values())


class LoanManager:
    """Manages checkouts and returns using composition."""

    def __init__(self, catalog: LibraryCatalog):
        self._catalog = catalog
        self._checkouts = {}

    def checkout_item(self, user_name: str, isbn: str):
        """
        Checks out an item using polymorphic behavior.
        The system handles different item types uniformly.
        """
        item = self._catalog.get_item(isbn)
        if not item:
            return "Error: Item not found."

        # Polymorphic call: Same method produces different results (loan days)
        due_date, message = item.check_out()

        if due_date:
            self._checkouts[isbn] = {"user": user_name, "due_date": due_date}
            return f"{message} User: {user_name}"
        return message

    def return_item(self, isbn: str, days_late: int = 0, fee_per_day: float = 0.50):
        """Handles return and fee calculation."""
        item = self._catalog.get_item(isbn)
        if not item:
            return "Error: Item not found."
        if item.available:
            return "Item was not checked out."

        item.return_item()
        user_name = self._checkouts.pop(isbn, {}).get("user", "Unknown")

        fee = max(days_late * fee_per_day, 0)
        return f"{user_name} returned '{item.title}'. Late fee: ${fee:.2f}"


# =================================================================
# USAGE DEMONSTRATION
# =================================================================

if __name__ == "__main__":
    print("--- 🏛️ LIBRARY SYSTEM INITIALIZATION ---")

    # 1. Initialize Composition classes
    catalog = LibraryCatalog()
    loan_manager = LoanManager(catalog)

    # 2. Instantiate and Add Derived Item Types
    book1 = Book("1984", "9780451524935", 1949, "George Orwell", "Dystopian")
    dvd1 = DVD("2001: A Space Odyssey", "D9999", 1968, "Stanley Kubrick")
    ebook1 = EBook("Python Guide", "E8888", 2023, "G. Programmer", 10.5)

    catalog.add_item(book1)
    catalog.add_item(dvd1)
    catalog.add_item(ebook1)

    print("\n--- 3. POLYMORPHIC CHECKOUT ---")

    # The LoanManager calls the generic 'check_out',
    # but the result (loan period) is specialized by the derived class.

    # Book (14 days loan)
    print(loan_manager.checkout_item("Alice", "9780451524935"))

    # DVD (3 days loan)
    print(loan_manager.checkout_item("Bob", "D9999"))

    # EBook (28 days loan)
    print(loan_manager.checkout_item("Charlie", "E8888"))

    print("\n--- 4. ITEM STATUS ---")
    for item in catalog.all_items:
        print(item)

    print("\n--- 5. RETURN & FEE CALCULATION ---")
    # Return Book 10 days late (Fee: 10 * $0.50 = $5.00)
    print(loan_manager.return_item("9780451524935", days_late=10))

    # Return DVD on time (Fee: $0.00)
    print(loan_manager.return_item("D9999", days_late=0))

    # Attempt to checkout the Book again (should succeed)
    print(loan_manager.checkout_item("Dave", "9780451524935"))

--- 🏛️ LIBRARY SYSTEM INITIALIZATION ---

--- 3. POLYMORPHIC CHECKOUT ---
📚 Book '1984' checked out. Due: 2025-12-04. User: Alice
📀 DVD '2001: A Space Odyssey' checked out. Due: 2025-11-23. User: Bob
📱 EBook 'Python Guide' checked out. Due: 2025-12-18. User: Charlie

--- 4. ITEM STATUS ---
[Checked Out] 📚 Book: 1984 by George Orwell (1949)
[Checked Out] 📀 DVD: 2001: A Space Odyssey directed by Stanley Kubrick (1968)
[Checked Out] 📱 EBook: Python Guide by G. Programmer (10.5MB)

--- 5. RETURN & FEE CALCULATION ---
Alice returned '1984'. Late fee: $5.00
Bob returned '2001: A Space Odyssey'. Late fee: $0.00
📚 Book '1984' checked out. Due: 2025-12-04. User: Dave
